In [4]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97
      # Set a custom testing threshold
    return DefaultPredictor(cfg)

def process_image(predictor, image_path):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Visualize the results
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Display the image
    result_image = out.get_image()[:, :, ::-1]
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        
        # Display the frame
        result_frame = out.get_image()[:, :, ::-1]
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path)

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_edge_combined/tank-and-ramp/mask_rcnn_R_101_FPN_3x/2024-07-01-16-00-38/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_edge_combined/tank-and-ramp/mask_rcnn_R_101_FPN_3x/2024-07-01-16-00-38/model_final.pth"
input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test"


predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder)


In [3]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import numpy as np

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97  # Set a custom testing threshold
    return DefaultPredictor(cfg)

def draw_quadrilaterals(image, outputs, deformation_threshold=0.05):
    masks = outputs["instances"].pred_masks.to("cpu").numpy()
    for mask in masks:
        # Find contours
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) == 0:
            continue
        
        for contour in contours:
            # Approximate the contour to a quadrilateral
            epsilon = deformation_threshold * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Ensure it's a quadrilateral (4 sides)
            if len(approx) == 4:
                # Draw the quadrilateral
                cv2.polylines(image, [approx], True, (0, 255, 0), 2)
                
    return image


def process_image(predictor, image_path):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Draw quadrilaterals and masks on the image
    result_image = draw_quadrilaterals(image, outputs)
    
    # Display the image
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        result_frame = draw_quadrilaterals(frame, outputs)
        
        # Display the frame
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path)

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_edge_combined/tank-and-ramp/mask_rcnn_R_101_FPN_3x/2024-07-01-16-00-38/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_edge_combined/tank-and-ramp/mask_rcnn_R_101_FPN_3x/2024-07-01-16-00-38/model_final.pth"
input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_edge_combined/tank-and-ramp-1/test"


predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder)
